<a href="https://colab.research.google.com/github/arthurvale/Perda-de-percurso-para-Redes-LoRaWAN-em-Ambientes-Compostos/blob/main/ETL_Dados_Medicao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bibliotecas

In [ ]:
import json
from google.colab import files
import struct
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

!pip install geopy
from geopy import distance

import numpy as np
import matplotlib.pyplot as plt

!pip install folium
import folium

from datetime import datetime
import matplotlib.dates as mdates



In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import math

# Importação de arquivo

In [ ]:
upload=files.upload()

Saving medicao_irlanda_alvaro.json to medicao_irlanda_alvaro (2).json


# Transformação de json

In [ ]:
with open('/content/medicao_irlanda_alvaro.json', 'r') as arquivo:
    dados = json.load(arquivo)

In [ ]:
#filtrando apenas as.up.data.forward
filter=[]
for d in dados:
  if d['name']=='as.up.data.forward':
    filter.append(d)

In [ ]:
filter2=[]
for dicionario in filter:


    if dicionario['data']['uplink_message']['decoded_payload']['type'] == 'Periodical Position (GPS)':

        val2 = hex(dicionario['data']['uplink_message']['decoded_payload']['val2'])[2:]
        if len(val2) < 2:
            val2 = '0'+val2
        val3 = hex(dicionario['data']['uplink_message']['decoded_payload']['val3'])[2:]
        if len(val3) < 2:
            val3 = '0'+val3
        val4 = hex(dicionario['data']['uplink_message']['decoded_payload']['val4'])[2:]
        if len(val4) < 2:
            val4 = '0'+val4
        val5 = hex(dicionario['data']['uplink_message']['decoded_payload']['val5'])[2:]
        if len(val5) < 2:
            val5 = '0'+val5

        val6 = hex(dicionario['data']['uplink_message']['decoded_payload']['val6'])[2:]
        if len(val6) < 2:
            val6 = '0'+val6
        val7 = hex(dicionario['data']['uplink_message']['decoded_payload']['val7'])[2:]
        if len(val7) < 2:
            val7 = '0'+val7
        val8 = hex(dicionario['data']['uplink_message']['decoded_payload']['val8'])[2:]
        if len(val8) < 2:
            val8 = '0'+val8
        val9 = hex(dicionario['data']['uplink_message']['decoded_payload']['val9'])[2:]
        if len(val9) < 2:
            val9 = '0'+val9


        longitude = val2+val3+val4+val5
        latitude = val6+val7+val8+val9

        longitude = struct.unpack('!f', bytes.fromhex(longitude))[0]
        latitude = struct.unpack('!f', bytes.fromhex(latitude))[0]

        dicionario['longitude'] = longitude
        dicionario['latitude'] = latitude

        filter2.append(dicionario)

In [ ]:
fliter3=[]

for a in filter2:
  a['consumed_airtime']=a['data']['uplink_message']['consumed_airtime']
  a['frequency']=a['data']['uplink_message']['settings']['frequency']
  a['spreading_factor']=a['data']['uplink_message']['settings']['data_rate']['lora']['spreading_factor']
  a['bandwidth']=a['data']['uplink_message']['settings']['data_rate']['lora']['bandwidth']
  a['coding_rate']=a['data']['uplink_message']['settings']['data_rate']['lora']['coding_rate']

  if len(a['data']['uplink_message']['rx_metadata'])==1:
    a['tempo'] = a['data']['uplink_message']['rx_metadata'][0]['received_at']
    a['snr'] = a['data']['uplink_message']['rx_metadata'][0]['snr']
    a['rssi'] = a['data']['uplink_message']['rx_metadata'][0]['rssi']
    a['channel_rssi'] = a['data']['uplink_message']['rx_metadata'][0]['channel_rssi']
    fliter3.append(a)

  else:
    for cont in range(len(a['data']['uplink_message']['rx_metadata'])):
      if (a['data']['uplink_message']['rx_metadata'][cont]['gateway_ids']['gateway_id']=='packetbroker') and (a['data']['uplink_message']['rx_metadata'][cont]['packet_broker']['forwarder_gateway_id']=='corkcoco'):
            a['tempo'] = a['data']['uplink_message']['rx_metadata'][cont]['received_at']
            a['snr'] = a['data']['uplink_message']['rx_metadata'][cont]['snr']
            a['rssi'] = a['data']['uplink_message']['rx_metadata'][cont]['rssi']
            a['channel_rssi'] = a['data']['uplink_message']['rx_metadata'][cont]['channel_rssi']
            fliter3.append(a)
fliter3[0]

{'name': 'as.up.data.forward',
 'time': '2023-12-19T12:41:34.123427480Z',
 'identifiers': [{'device_ids': {'device_id': 'eui-70b3d5a4d2110007',
    'application_ids': {'application_id': 'nimbus-lorawan-testbed'},
    'dev_eui': '70B3D5A4D2110007',
    'join_eui': '2C26C50211000003',
    'dev_addr': '260B2B0E'}}],
 'data': {'@type': 'type.googleapis.com/ttn.lorawan.v3.ApplicationUp',
  'end_device_ids': {'device_id': 'eui-70b3d5a4d2110007',
   'application_ids': {'application_id': 'nimbus-lorawan-testbed'},
   'dev_eui': '70B3D5A4D2110007',
   'join_eui': '2C26C50211000003',
   'dev_addr': '260B2B0E'},
  'correlation_ids': ['pba:uplink:01HJ11H0YGD974K0XM73HYX461'],
  'received_at': '2023-12-19T12:41:34.117749367Z',
  'uplink_message': {'session_key_id': 'AYyBjHrA4LGwgqjZMr+Cug==',
   'f_port': 8,
   'f_cnt': 724,
   'frm_payload': 'MMEIji5CT4toZYGPeg==',
   'decoded_payload': {'port': 8,
    'type': 'Periodical Position (GPS)',
    'val1': 48,
    'val10': 101,
    'val11': 129,
    'va

#Transformações e calculos no Dataframe

In [ ]:
dados = pd.DataFrame(fliter3)
chaves = ['longitude', 'latitude', 'consumed_airtime', 'frequency', 'spreading_factor', 'bandwidth',
                       'coding_rate', 'tempo', 'snr', 'rssi', 'channel_rssi']

dados_finalizados= dados[chaves]
dados_finalizados['altura_gtw']= 67
dados_finalizados['latitude_gtw']= 51.892567
dados_finalizados['longitude_gtw']= -8.508705
dados_finalizados

,longitude,latitude,consumed_airtime,frequency,spreading_factor,bandwidth,coding_rate,tempo,snr,rssi,channel_rssi,altura_gtw,latitude_gtw,longitude_gtw
0,-8.534712,51.886139,0.061696s,868300000,7,125000,4/5,2023-12-19T12:41:33.902264048Z,-2.5,-115,-115,67,51.892567,-8.508705
1,-8.534475,51.886147,0.061696s,868100000,7,125000,4/5,2023-12-19T12:41:21.552299923Z,0.5,-115,-115,67,51.892567,-8.508705
2,-8.533717,51.886177,0.061696s,868500000,7,125000,4/5,2023-12-19T12:41:01.792946555Z,-0.5,-117,-117,67,51.892567,-8.508705
3,-8.533300,51.886177,0.061696s,868300000,7,125000,4/5,2023-12-19T12:40:34.200615827Z,3.0,-115,-115,67,51.892567,-8.508705
4,-8.533100,51.886185,0.061696s,868100000,7,125000,4/5,2023-12-19T12:40:19.793905906Z,3.0,-116,-116,67,51.892567,-8.508705
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
526,-8.506374,51.880413,0.066816s,868300000,7,125000,4/5,2023-12-19T08:18:47.848968728Z,0.5,-106,-106,67,51.892567,-8.508705
527,-8.506102,51.879505,0.113152s,867300000,8,125000,4/5,2023-12-19T08:18:42.031011092Z,10.0,-98,-98,67,51.892567,-8.508705
528,-8.505987,51.878948,0.113152s,868300000,8,125000,4/5,2023-12-19T08:18:13.958449444Z,2.2,-111,-111,67,51.892567,-8.508705
529,-8.505971,51.878895,0.123392s,867100000,8,125000,4/5,2023-12-19T08:18:07.797497653Z,8.8,-113,-113,67,51.892567,-8.508705


In [ ]:
dados_finalizados

,longitude,latitude,consumed_airtime,frequency,spreading_factor,bandwidth,coding_rate,tempo,snr,rssi,channel_rssi,altura_gtw,latitude_gtw,longitude_gtw
0,-8.534712,51.886139,0.061696s,868300000,7,125000,4/5,2023-12-19T12:41:33.902264048Z,-2.5,-115,-115,67,51.892567,-8.508705
1,-8.534475,51.886147,0.061696s,868100000,7,125000,4/5,2023-12-19T12:41:21.552299923Z,0.5,-115,-115,67,51.892567,-8.508705
2,-8.533717,51.886177,0.061696s,868500000,7,125000,4/5,2023-12-19T12:41:01.792946555Z,-0.5,-117,-117,67,51.892567,-8.508705
3,-8.533300,51.886177,0.061696s,868300000,7,125000,4/5,2023-12-19T12:40:34.200615827Z,3.0,-115,-115,67,51.892567,-8.508705
4,-8.533100,51.886185,0.061696s,868100000,7,125000,4/5,2023-12-19T12:40:19.793905906Z,3.0,-116,-116,67,51.892567,-8.508705
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
526,-8.506374,51.880413,0.066816s,868300000,7,125000,4/5,2023-12-19T08:18:47.848968728Z,0.5,-106,-106,67,51.892567,-8.508705
527,-8.506102,51.879505,0.113152s,867300000,8,125000,4/5,2023-12-19T08:18:42.031011092Z,10.0,-98,-98,67,51.892567,-8.508705
528,-8.505987,51.878948,0.113152s,868300000,8,125000,4/5,2023-12-19T08:18:13.958449444Z,2.2,-111,-111,67,51.892567,-8.508705
529,-8.505971,51.878895,0.123392s,867100000,8,125000,4/5,2023-12-19T08:18:07.797497653Z,8.8,-113,-113,67,51.892567,-8.508705


In [ ]:
dados_finalizados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 531 entries, 0 to 530
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   longitude         531 non-null    float64
 1   latitude          531 non-null    float64
 2   consumed_airtime  531 non-null    object 
 3   frequency         531 non-null    object 
 4   spreading_factor  531 non-null    int64  
 5   bandwidth         531 non-null    int64  
 6   coding_rate       531 non-null    object 
 7   tempo             531 non-null    object 
 8   snr               531 non-null    float64
 9   rssi              531 non-null    int64  
 10  channel_rssi      531 non-null    int64  
 11  altura_gtw        531 non-null    int64  
 12  latitude_gtw      531 non-null    float64
 13  longitude_gtw     531 non-null    float64
dtypes: float64(5), int64(5), object(4)
memory usage: 58.2+ KB


## Transformação das colunas de data

In [ ]:
dados_finalizados['intervalo_tempo'] = pd.to_timedelta(dados_finalizados['consumed_airtime'])
dados_finalizados['horario_receb'] = pd.to_datetime(dados_finalizados['tempo'])
dados_finalizados['horario_envio'] = dados_finalizados['horario_receb'] - dados_finalizados['intervalo_tempo']
dados_finalizados.drop(['consumed_airtime'], axis=1, inplace=True)

In [ ]:
dados_finalizados['latitude_lora']=dados_finalizados['latitude']
dados_finalizados['longitude_lora']=dados_finalizados['longitude']
dados_finalizados.drop(['latitude', 'longitude'], axis=1, inplace=True)

## Calculo da distancia Real

In [ ]:
def calcular_distancia(row):
    coord_origem = (row['latitude_gtw'], row['longitude_gtw'])
    coord_destino = (row['latitude_lora'], row['longitude_lora'])
    return distance.distance(coord_origem, coord_destino).meters

dados_finalizados['distancia_horizontal'] = dados_finalizados.apply(calcular_distancia, axis=1)
dados_finalizados['distancia_real'] = np.sqrt(dados_finalizados['altura_gtw']**2 + dados_finalizados['distancia_horizontal']**2)
dados_finalizados.reset_index(drop=True, inplace=True)

In [ ]:
dados_finalizados[['distancia_real', 'rssi', 'distancia_horizontal']].describe()

,distancia_real,rssi,distancia_horizontal
count,531.000000,531.000000,531.000000
mean,1388.762328,-108.084746,1386.536375
std,630.326909,8.983337,631.667541
min,169.990227,-129.000000,156.229566
25%,901.586513,-115.500000,899.093566
50%,1363.103585,-109.000000,1361.455979
75%,1918.530998,-101.000000,1917.360732
max,2553.854445,-79.000000,2552.975426


In [ ]:
dados_finalizados['frequency'] = pd.to_numeric(dados_finalizados['frequency'], errors='coerce')

print(dados_finalizados['frequency'].value_counts())
dados_finalizados['frequency'].describe()

frequency
868500000    129
868100000    123
868300000    117
867700000     44
867300000     42
867100000     36
867900000     35
867500000      5
Name: count, dtype: int64


count    5.310000e+02
mean     8.680582e+08
std      4.289343e+05
min      8.671000e+08
25%      8.679000e+08
50%      8.681000e+08
75%      8.683000e+08
max      8.685000e+08
Name: frequency, dtype: float64

In [ ]:
dados_finalizados.columns

Index(['frequency', 'spreading_factor', 'bandwidth', 'coding_rate', 'tempo',
       'snr', 'rssi', 'channel_rssi', 'altura_gtw', 'latitude_gtw',
       'longitude_gtw', 'intervalo_tempo', 'horario_receb', 'horario_envio',
       'latitude_lora', 'longitude_lora', 'distancia_horizontal',
       'distancia_real'],
      dtype='object')

## Verificação de linhas duplicadasd e preparação para o arquivo finalm seleção das colunas, etc.

In [ ]:
arquivo_final = dados_finalizados[['latitude_lora', 'longitude_lora', 'latitude_gtw', 'longitude_gtw',
                                   'distancia_real', 'intervalo_tempo', 'horario_receb', 'rssi', 'snr']]
arquivo_final['rssi'] = arquivo_final['rssi'].astype(float)
arquivo_final['horario_receb'] = arquivo_final['horario_receb'].dt.tz_localize(None)
arquivo_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 531 entries, 0 to 530
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype          
---  ------           --------------  -----          
 0   latitude_lora    531 non-null    float64        
 1   longitude_lora   531 non-null    float64        
 2   latitude_gtw     531 non-null    float64        
 3   longitude_gtw    531 non-null    float64        
 4   distancia_real   531 non-null    float64        
 5   intervalo_tempo  531 non-null    timedelta64[ns]
 6   horario_receb    531 non-null    datetime64[ns] 
 7   rssi             531 non-null    float64        
 8   snr              531 non-null    float64        
dtypes: datetime64[ns](1), float64(7), timedelta64[ns](1)
memory usage: 37.5 KB


In [ ]:
linhas_duplicadas = arquivo_final.duplicated(keep=False)
if linhas_duplicadas.any():
    print("Linhas duplicadas encontradas:")
    print(arquivo_final[linhas_duplicadas])
else:
    print("Não há linhas duplicadas com todos os dados duplicados.")
arquivo_final.drop_duplicates(inplace=True)
arquivo_final.reset_index(drop=True, inplace=True)
linhas_duplicadas = arquivo_final.duplicated(keep=False)
if linhas_duplicadas.any():
    print("Linhas duplicadas encontradas:")
    print(arquivo_final[linhas_duplicadas])
else:
    print("Não há linhas duplicadas com todos os dados duplicados.")

Linhas duplicadas encontradas:
     latitude_lora  longitude_lora  latitude_gtw  longitude_gtw  \
398      51.898865       -8.476294     51.892567      -8.508705   
399      51.898800       -8.476370     51.892567      -8.508705   
400      51.898804       -8.476431     51.892567      -8.508705   
401      51.898865       -8.476294     51.892567      -8.508705   
402      51.898800       -8.476370     51.892567      -8.508705   
403      51.898804       -8.476431     51.892567      -8.508705   

     distancia_real        intervalo_tempo                 horario_receb  \
398     2339.530614 0 days 00:00:00.061696 2023-12-19 09:15:32.766868655   
399     2332.368521 0 days 00:00:00.061696 2023-12-19 09:13:45.065739515   
400     2328.485559 0 days 00:00:00.061696 2023-12-19 09:13:37.666960518   
401     2339.530614 0 days 00:00:00.061696 2023-12-19 09:15:32.766868655   
402     2332.368521 0 days 00:00:00.061696 2023-12-19 09:13:45.065739515   
403     2328.485559 0 days 00:00:00.061696 

#Verificação da integridade para finalizar a exportação para arquivo .csv (drive)

In [ ]:
!pip install pandera

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.9/251.9 kB 7.4 MB/s eta 0:00:00


In [ ]:
import pandera as pa

In [ ]:
arquivo_final.dtypes

In [ ]:
schema = pa.DataFrameSchema({'latitude_lora': pa.Column(pa.Float),
                             'longitude_lora': pa.Column(pa.Float),
                             'latitude_gtw': pa.Column(pa.Float),
                             'longitude_gtw': pa.Column(pa.Float),
                             'distancia_real': pa.Column(pa.Float),
                             'intervalo_tempo': pa.Column(pa.Timedelta),
                             'horario_receb':pa.Column(pa.DateTime),
                             'rssi': pa.Column(pa.Float),
                             'snr': pa.Column(pa.Float)
                             })

In [ ]:
schema.validate(arquivo_final)

arquivo_final.to_csv('/content/drive/MyDrive/Colab Notebooks/Dados - irlanda/Dados Irlanda Tratados.csv', index=False,
                     date_format='%Y-%m-%d %H:%M:%S')

